In [7]:
from ase import Atoms
from ase.build import bulk
from ase.constraints import ExpCellFilter, StrainFilter
from ase.optimize import LBFGS, FIRE
import pfp_api_client
from pfp_api_client.pfp.calculators.ase_calculator import ASECalculator
from pfp_api_client.pfp.estimator import Estimator, EstimatorCalcMode
print(f"pfp_api_client: {pfp_api_client.__version__}")
import numpy as np
import pandas as pd
import requests
import pprint
import sys
import string
import json
import io
import copy


pfp_api_client: 1.8.0


In [9]:
import requests
import pprint
import sys
import string
import json

GRAPHQL = 'http://api.catalysis-hub.org/graphql'
def fetch(query):
    return requests.get(
        GRAPHQL, {'query': query}
    ).json()['data']
def reactions_from_dataset(pub_id, page_size=40):
    reactions = []
    has_next_page = True
    start_cursor = ''
    page = 0
    while has_next_page:
        data = fetch("""{{
      reactions(pubId: "{pub_id}", first: {page_size}, after: "{start_cursor}") {{
        totalCount
        pageInfo {{
          hasNextPage
          hasPreviousPage
          startCursor
          endCursor 
        }}  
        edges {{
          node {{
            Equation
            reactants
            products
            reactionEnergy
            reactionSystems {{
              name
              systems {{
                energy
                InputFile(format: "json")
              }}
            }}  
          }}  
        }}  
      }}    
    }}""".format(start_cursor=start_cursor,
                 page_size=page_size,
                 pub_id=pub_id,
                ))
        has_next_page = data['reactions']['pageInfo']['hasNextPage']
        start_cursor = data['reactions']['pageInfo']['endCursor']
        page += 1
        print(has_next_page, start_cursor, page_size * page, data['reactions']['totalCount'])
        reactions.extend(map(lambda x: x['node'], data['reactions']['edges']))
    return reactions
dataset_name_is="PatelGeneralizable2021"
raw_reactions7 = reactions_from_dataset(dataset_name_is) # sac 
import json
import os
def data2json(raw_reactions, dataset_name):
    raw_reactions_json = {"raw_reactions": raw_reactions}
    filename = f"{dataset_name}.json"
    path = os.path.join(os.getcwd(), filename)
    with open(path, 'w') as f:
        json.dump(raw_reactions_json, f)
data2json(raw_reactions7,dataset_name_is)
path_json = os.path.join(os.getcwd(), dataset_name_is+".json")
with open(path_json, 'r') as f:
  data = json.load(f)
pierre=data['raw_reactions']


ConnectionError: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))

In [6]:
def aseify_reactions(reactions):
    for i, reaction in enumerate(reactions):
        for j, _ in enumerate(reactions[i]['reactionSystems']):
            with io.StringIO() as tmp_file:
                system = reactions[i]['reactionSystems'][j].pop('systems')
                tmp_file.write(system.pop('InputFile'))
                tmp_file.seek(0)
                atoms = ase.io.read(tmp_file, format='json')
            calculator = ase.calculators.singlepoint.SinglePointCalculator(
                atoms,
                energy=system.pop('energy')
            )
            atoms.set_calculator(calculator)
            #print(atoms.get_potential_energy())
            reactions[i]['reactionSystems'][j]['atoms'] = atoms
        # flatten list further into {name: atoms, ...} dictionary
        reactions[i]['reactionSystems'] = {x['name']: x['atoms']
                                          for x in reactions[i]['reactionSystems']}